In [11]:
# Importing necessary modules

import os
import math
import pandas as pd
import numpy as np
import glob
from itertools import cycle
import matplotlib.pyplot as plt
from scipy import stats
import shutil



# Identifying directory of lab sheets for quality screening

path = str(glob.os.getcwd())

user = path.split('\\')[2]


sheet_name = input('Provide the name of the file you wish to check.  (do NOT include .csv in file name)')

sheet_name = sheet_name + '.csv'

filea = sheet_name.split('_')[-6]
fileb = sheet_name.split('_')[-5]
filec = sheet_name.split('_')[-4]

folder = "Lab_Sheets/" + str(filea) + '_' + str(fileb) + '_' + str(filec) + '/'

datasheet = '/Users/' + user + '/Desktop/StandardLab/' + folder

glob.os.chdir(datasheet)


# Reading working file as dataframe in pandas

lab_data = pd.read_csv(sheet_name)


# Pulling parts of name to identify sheet for naming/moving

title = sheet_name.split('_')[2]


print(sheet_name + ' has the following properties:')
### SCREENING FOR PRESENCE OF DATA            



# Checking for data - skipping remainder of code (breaking loop) if there is no data

if not np.isnan(lab_data.Absorbance[4]):

#                     break


#                 # Continuing if data is found

#                 else:

    # Identifying sheet




### SCREENING BLANKS                        



    # Creating empty lists to append with 'blanks' information

    blanks_abs = []


    # Identifying blank absorbances based on 'Type' column

    for n in range(len(lab_data)):
        if lab_data.Type[n] == 'B':
            blanks_abs.append(float(lab_data.Absorbance[n]))


    # Creating a conditional to filter blank absorbances outside of acceptable range

    good_blanks = [x for x in blanks_abs if x < 0.1 and x > -0.1]


    # If the list of acceptable blanks has fewer than three values, set must be repeated

    if len(good_blanks) <= 2 :


        print("  - Blank readings are questionable:" + str(blank_abs))



    # If the list of acceptable blanks has three good values, set is acceptable   

    else:


        print("  - Acceptable blank readings")


### SCREENING CURVES

## code will check if curve can be 
## adjusted to produce and acceptable r^2






        # Creating empty lists to append with curve information

        curve_absorbance = []
        curve_concentration = []


        # Identifying standard absorbances based on 'Type' column

        for n in range(len(lab_data)):
            if lab_data.Type[n] == 'C':


                # Appending calibration data to appropriate list for curve analysis

                curve_absorbance.append(float(lab_data.Absorbance[n]))
                curve_concentration.append(float(lab_data.Sample_ID[n]))


        # Calculating r^2 value for two calibration data lists with lin.regress module

        slope, intercept, r_value1, p_value, std_err = stats.linregress(curve_absorbance, curve_concentration)


        # Creating conditional for screening based on r^2 value... using a matrix to calculate optional r^2 values based
        # on removal of one curve point

        if r_value1**2 < 0.95:

            # Creating pandas dataframe for analysis

            columns = [['Conc', 'Abs']]

            curve = pd.DataFrame(columns = columns)


            # Assigning values to concentration and absorbance columns

            curve['Conc'] = curve_concentration
            curve['Abs'] = curve_absorbance


            # Defining the size of the possible removal matrix

            points = len(curve_concentration) # Total number of datapoints


            # Creating a matrix via a numpy array 

            array = np.identity(points)


            # Using a for-loop to generate new columns in the new pandas df containing values from decision matrix

            for i in range(array.shape[1]):

                curve[str(i)] = array[:,i]


            # Appending 0's and 1's to df for all combinations of 1 and 2-point removal

            for i in range(1,points):

                array = np.identity(i)
                array = np.vstack([array, [1]*i])

                if array.shape[0] != 12:
                    for j in range(12 - array.shape[0]):
                        array = np.vstack([array, [0]*array.shape[1]])

                    for k in range(array.shape[1]):

                        curve[str(i)+str(k)+str(k)] = array[:,k]


            # Assigning column names and creating pandas dataframe to hold/analyze "new" curve parameters

            columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

            columns = []
            slopes = []
            intercepts = []
            rvals = []
            pvals = []
            stderrs = []

            parameters = pd.DataFrame(columns = columns)


            # Running conditional by column in new df (ONLY CONSIDERING THE COLUMNS OF THE MATRIX IN WHICH ONE POINT IS REMOVED)

            for column in curve.iloc[:, 2:14]:

                columns.append(column)


                # Creating iterations counter and creating lists to append in for-loop

                count = 0
                conc = []
                absorb = []


                # Running conditional by row in column

                for i in curve[column]: 


                    # Conditional "removing" rows of column by not appending values to respective lists if value is 1

                    if i < 1:
                        conc.append(curve.iloc[count,0])
                        absorb.append(curve.iloc[count,1])


                    # Appending counter to reflect the next row for iteration

                    count = count + 1


                # Assigning variables to outputs from stats.linregress() function and appending list with column values

                slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

                slopes.append(slope)
                intercepts.append(inter)
                rvals.append(r_val**2)
                pvals.append(p_val)
                stderrs.append(std)


            # Assigning regression values to columns in parameters dataframe    

            parameters['Column'] = columns
            parameters['Slope'] = slopes
            parameters['Intercept'] = intercepts
            parameters['R_Sq'] = rvals
            parameters['P_Val'] = pvals
            parameters['StdErr'] = stderrs


            ## TO SEE THE PARAMETERS AND PRINT THE R^2 CALCULATED BY REMOVAL OF ONE POINT OF THE CURVE:

            #print(parameters['R_Sq'].max())
            #print(parameters)


            # Creating a second conditional for r^2 based on a matrix that calculates optional r^2 values based
            # on removal of two curve points ... THIS CODE WILL ONLY BE RUN IF THE HIGHEST POSSIBLE R^2 VALUE IN
            # THE PREVIOUS DATAFRAME IS LESS THAN 0.95

            if parameters['R_Sq'].max() < 0.95:


                # OVERWRITING previous column names and dataframe to hold/analyze "new" secondary curve parameters

                columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

                columns = []
                slopes = []
                intercepts = []
                rvals = []
                pvals = []
                stderrs = []

                parameters = pd.DataFrame(columns = columns)


                # Running conditional by column in new df (ONLY CONSIDERING THE COLUMNS OF THE MATRIX IN WHICH TWO POINTS ARE REMOVED)

                for column in curve.iloc[:, 14:]:

                    columns.append(column)


                    # Creating iterations counter and creating lists to append in for-loop

                    count = 0
                    conc = []
                    absorb = []


                    # Running conditional by row in column

                    for i in curve[column]: 


                        # Conditional "removing" rows of column by not appending values to respective lists if value is 1

                        if i < 1:
                            conc.append(curve.iloc[count,0])
                            absorb.append(curve.iloc[count,1])


                        # Appending counter to reflect the next row for iteration    

                        count = count + 1


                    # Assigning variables to outputs from stats.linregress() function and appending list with column values    

                    slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

                    slopes.append(slope)
                    intercepts.append(inter)
                    rvals.append(r_val**2)
                    pvals.append(p_val)
                    stderrs.append(std)

                # Assigning regression values to columns in parameters dataframe 

                parameters['Column'] = columns
                parameters['Slope'] = slopes
                parameters['Intercept'] = intercepts
                parameters['R_Sq'] = rvals
                parameters['P_Val'] = pvals
                parameters['StdErr'] = stderrs



            print("  - A calibration curve r-squared value of " + str(r_value1**2))
            print("  - A CORRECTED curve r-squared value of " + str(parameters.R_Sq.max()))                    

### SCREENING FOR SAMPLE ORIGINAL/DUPLICATE VARIATION                                            



        # If the calibration data is acceptable:                 

        else: 

            # Defining which rows contain sample data

            print("  - A calibration curve r-squared value of " + str(r_value1**2))

        sample_data = lab_data[lab_data.Type != 'C']
        sample_data = sample_data[sample_data.Type != 'B']


        # Creating new dataframe with only ID & absorbance

        sample_data_ab = sample_data[['Sample_ID', 'Absorbance']]


        # Grouping sample data based on unique ID

        sample_data_g = sample_data_ab.groupby('Sample_ID')


        # Merging and creating a new dataframe for original/duplicate variance values (max-min/max)

        sample_data_var = sample_data_ab.merge((abs((sample_data_g.max() - sample_data_g.min())) / (sample_data_g.max())), on='Sample_ID')


        # Create conditional based on variance column

        U_Re_Run = []
        N_Re_Run = []

        for i, row in sample_data_var.iterrows():
            if float(row['Absorbance_y']) > 0.1:


                # Adding sample ID information to appropriate list if variance is unacceptable

                if title == "URE":
                    U_Re_Run.append(row['Sample_ID'])
                elif title == "NIT":
                    N_Re_Run.append(row['Sample_ID'])

        U_Run_List = set(U_Re_Run)    
        N_Run_List = set(N_Re_Run)

        if title == "URE":

            print('  - Greater than 10% variance in ' + str(len(U_Run_List)) + " sample(s)")

        elif title == 'NIT':

            print('  - Greater than 10% variance in ' + str(len(N_Run_List)) + " sample(s)")

Provide the name of the file you wish to check.  (do NOT include .csv in file name) A_1_URE_MS_GH_LI_06_25_18


A_1_URE_MS_GH_LI_06_25_18.csv has the following properties:
  - Acceptable blank readings
  - A calibration curve r-squared value of 0.899899494585602
  - A CORRECTED curve r-squared value of 0.9527330154900056
  - Greater than 10% variance in 1 sample(s)
